In [ ]:
import optuna
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import lightgbm as lgb
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [ ]:
ecommerce_data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding='ISO-8859-1', dtype={'CustomerID': str})

In [ ]:
ecommerce_data.head()

## Data Description

In [ ]:
def check_data(dataframe, head=5):
    print(' SHAPE '.center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(' TYPES '.center(70,'-'))
    print(dataframe.dtypes)
    print(' HEAD '.center(70,'-'))
    print(dataframe.head(head))
    print(' TAIL '.center(70,'-'))
    print(dataframe.tail(head))
    print(' MISSING VALUES '.center(70,'-'))
    print(dataframe.isnull().sum())
    print(' DUPLICATED VALUES '.center(70,'-'))
    print(dataframe.duplicated().sum())
    print(' QUANTILES '.center(70,'-'))
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1], numeric_only=True).T)
    
check_data(ecommerce_data)

In [ ]:
world_map = ecommerce_data[['CustomerID', 'InvoiceNo', 'Country']
              ].groupby(['CustomerID', 'InvoiceNo', 'Country']
                       ).count().reset_index(drop = False)
countries = world_map['Country'].value_counts()
data = dict(type='choropleth',
            locations = countries.index,
            locationmode = 'country names',
            z = countries,
            text = countries.index,
            colorbar = {'title':'Orders'},
            colorscale='Viridis',
            reversescale = False)

layout = dict(title={'text': "Number of Orders by Countries",
                     'y':0.9,
                     'x':0.5,
                     'xanchor': 'center',
                     'yanchor': 'top'},
              geo = dict(resolution = 50,
                         showocean = True,
                         oceancolor = "LightBlue",
                         showland = True,
                         landcolor = "whitesmoke",
                         showframe = True),
             template = 'plotly_white',
             height = 700,
             width = 1000)

choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate = False)

In [ ]:
ecommerce_data.describe()

**Missing Data**

In [ ]:
missing_percentage = ecommerce_data.isnull().sum() / ecommerce_data.shape[0] * 100
missing_percentage

In [ ]:
ecommerce_data['Description'].isnull().sum()

In [ ]:
ecommerce_data['CustomerID'].isnull().sum()

In [ ]:
ecommerce_data.loc[ecommerce_data['CustomerID'].isnull(), ['UnitPrice', 'Quantity']].describe()

Removing data with null CustomerID and Description because they can have bad influence on the forecast.

In [ ]:
data = ecommerce_data.loc[(ecommerce_data['CustomerID'].isnull()==False) & (ecommerce_data['Description'].isnull()==False)].copy()

In [ ]:
data.describe()

In [ ]:
data[(data['Quantity'] == -80995) | (data['Quantity'] == 80995)]

In the InvoiceNo we can find cancelled transactions, that are also don't influence sales forecast, because reasons are unknown and they are hard to predict. So, we can just remove them.

In [ ]:
data['IsCancelled']=np.where(data['InvoiceNo'].apply(lambda no: no[0]=="C"), True, False)
data['IsCancelled'].value_counts() / data.shape[0] * 100

In [ ]:
data = data.loc[data['IsCancelled']==False].copy()
data = data.drop('IsCancelled', axis=1)

In [ ]:
# removing outlier
data = data[data['Quantity'] != 80995]

In [ ]:
data.describe(percentiles=[.25, .5, .75, .99])

### The time period

In [ ]:
data["InvoiceDate"] = pd.to_datetime(data['InvoiceDate'], cache=True)
print('Start date {}'.format(data['InvoiceDate'].min()))
print('End date {}'.format(data['InvoiceDate'].max()))
print('Period covered {}'.format(data['InvoiceDate'].max() - data['InvoiceDate'].min()))

### Stockecodes

In [ ]:
stockcode_counts = data['StockCode'].value_counts().sort_values(ascending=False)
fig, ax = plt.subplots(1,1,figsize=(20,5))
sns.barplot(x = stockcode_counts.iloc[0:20].index,
            y = stockcode_counts.iloc[0:20].values,
            ax = ax, palette="Spectral")
ax.set_ylabel("Counts")
ax.set_xlabel("Stockcode")
ax.set_title("Which stockcodes are most common?");

In [ ]:
def count_numeric_chars(l):
    return sum(1 for c in l if c.isdigit())

data['StockCodeLength'] = data['StockCode'].apply(lambda l: len(l))
data['nNumericStockCode'] = data['StockCode'].apply(lambda l: count_numeric_chars(l))

In [ ]:
data['StockCodeLength'].value_counts()

In [ ]:
data['nNumericStockCode'].value_counts()

In [ ]:
data.loc[data['StockCodeLength'] > 5]['StockCode'].value_counts()

In [ ]:
data.loc[data['nNumericStockCode'] < 5]['Description'].value_counts()

As we can see the transaction with the stock codes mentioned above won't help with stock prediction. So we can remove them and also replace Stock Codes for the rest with 5 numbers Stocke Codes, because additional letter can make only difference in colors.

In [ ]:
data = data.loc[(data['nNumericStockCode'] == 5)].copy()

In [ ]:
data['StockCode'] = data['StockCode'].apply(lambda x: x[:5])

In [ ]:
data = data.drop(['nNumericStockCode', 'StockCodeLength'], axis=1)

### Descriptions

In [ ]:
description_counts = data['Description'].value_counts().sort_values(ascending=False).iloc[0:30]
plt.figure(figsize=(20,5))
sns.barplot(x=description_counts.index, y=description_counts.values, palette="flare")
plt.ylabel("Counts")
plt.title("Which product descriptions are most common?");
plt.xticks(rotation=90);

In [ ]:
def count_upper_chars(l):
    return sum(1 for c in l if c.isupper())

data['UpCharsInDescription'] = data['Description'].apply(lambda l: count_upper_chars(l))

In [ ]:
data.loc[data['UpCharsInDescription'] <=5]['Description'].value_counts()

We can remove the items because they aren't products and it's quite complicated to predict them correectly. 

In [ ]:
data = data.loc[data['UpCharsInDescription'] > 5].copy()

In [ ]:
data = data.drop(['UpCharsInDescription'], axis=1)

### Customers

In [ ]:
customer_counts = data['CustomerID'].value_counts().sort_values(ascending=False).iloc[0:20] 
plt.figure(figsize=(20,5))
sns.barplot(x=customer_counts.index, y=customer_counts.values, order=customer_counts.index)
plt.ylabel("Counts")
plt.xlabel("CustomerID")
plt.title("Which customers are most common?");

As we saw on the map far above the retailer sells almost all products in the UK. Let's find a percentage. 

In [ ]:
data.loc[data['Country']=='United Kingdom'].shape[0] / data.shape[0] * 100

In [ ]:
# data_UK = data.loc[data['Country']=='United Kingdom'].copy()

### Unit Prices

In [ ]:
data['UnitPrice'].describe(percentiles=[.25, .5, .75, .99])

In [ ]:
data.loc[data['UnitPrice'] == 0].sort_values(by='Quantity', ascending=False).head()

In [ ]:
data = data.loc[data['UnitPrice'] > 0].copy()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(data['UnitPrice'], ax=ax[0], kde=False, color="red")
sns.distplot(np.log(data['UnitPrice']), ax=ax[1], bins=20, color="tomato", kde=False)
ax[1].set_xlabel("Log-Unit-Price");

In [ ]:
print(f'Lowest value: {np.exp(-2)}')
print(f'Highest value: {np.exp(3)}')

Let's keep only the data in the range.

In [ ]:
data = data.loc[(data['UnitPrice'] > 0.1) & (data['UnitPrice'] < 20)].copy()

### Quantities

In [ ]:
data['Quantity'].describe(percentiles=[.25, .5, .75, .99])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.distplot(data['Quantity'], ax=ax[0], kde=False, color="limegreen");
sns.distplot(np.log(data['Quantity']), ax=ax[1], bins=20, kde=False, color="limegreen");
ax[0].set_title("Quantity distribution")
ax[0].set_yscale("log")
ax[1].set_title("Log-Quantity distribution")
ax[1].set_xlabel("Natural-Log Quantity");

In [ ]:
print(f'Lowest value: {np.exp(0)}')
print(f'Highest value: {np.exp(4)}')

In [ ]:
data = data.loc[data['Quantity'] < 55].copy()

### Daily products sales

Creating additional features for predictions.

In [ ]:
data['Revenue'] = data['Quantity'] * data['UnitPrice']

data['Year'] = data['InvoiceDate'].dt.year
data['Quarter'] = data['InvoiceDate'].dt.quarter
data['Month'] = data['InvoiceDate'].dt.month
data['Week'] = data['InvoiceDate'].dt.week
data['Weekday'] = data['InvoiceDate'].dt.weekday
data['Day'] = data['InvoiceDate'].dt.day
data['DayOfYear'] = data['InvoiceDate'].dt.dayofyear
data['Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])

In [ ]:
grouped_features = ['Date', 'Year', 'Quarter','Month', 'Week', 'Weekday', 'DayOfYear', 'Day', 'StockCode']
daily_data = pd.DataFrame(data.groupby(grouped_features)['Quantity'].sum(),
                          columns=['Quantity'])
daily_data['Revenue'] = data.groupby(grouped_features)['Revenue'].sum()
daily_data = daily_data.reset_index()
daily_data.head(5)

In [ ]:
daily_data.loc[:, ['Quantity', 'Revenue']].describe()

We still have the outliers we need to remove.

In [ ]:
low_quantity = daily_data['Quantity'].quantile(0.01)
high_quantity = daily_data['Quantity'].quantile(0.99)
print((low_quantity, high_quantity))

In [ ]:
low_revenue = daily_data['Revenue'].quantile(0.01)
high_revenue = daily_data['Revenue'].quantile(0.99)
print((low_revenue, high_revenue))

In [ ]:
samples = daily_data.shape[0]

Let's only use target ranges data that are occupied by 90 % of the data entries, because we don't know the reason of the outliers and they can influence our model.

In [ ]:
daily_data = daily_data.loc[
    (daily_data['Quantity'] >= low_quantity) & (daily_data['Quantity'] <= high_quantity)]
daily_data = daily_data.loc[
    (daily_data['Revenue'] >= low_revenue) & (daily_data['Revenue'] <= high_revenue)]

In [ ]:
print(f'We lost {samples - daily_data.shape[0]} entries')

### Daily sales prediction

In [ ]:
daily_data

In [ ]:
def create_features(df, lags = [28], wins = [7,28]):
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[['StockCode','Quantity']].groupby('StockCode')['Quantity'].shift(lag).fillna(-1)

    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            df[f"rmean_{lag}_{win}"] = df[['Quantity', lag_col]].groupby('Quantity')[lag_col].transform(lambda x : x.rolling(win).mean()).fillna(-1)
        
    return df

In [ ]:
daily_data = create_features(daily_data)

In [ ]:
daily_data

In [ ]:
daily_data['StockCode'] = daily_data['StockCode'].astype(np.int64)

In [ ]:
daily_data.info()

In [ ]:
# split into train and test
cutoff = daily_data['Date'].max() - pd.to_timedelta(28, unit = 'D')
X_train = daily_data.loc[daily_data['Date'] < cutoff].copy()
X_val = daily_data.loc[daily_data['Date'] >= cutoff].copy()

In [ ]:
y_train = X_train['Quantity'].copy()
y_val = X_val['Quantity'].copy()

X_train.drop(['Quantity','Date'], axis = 1, inplace = True)
X_val.drop(['Quantity', 'Date'], axis = 1, inplace = True)

In [ ]:
train_df = lgb.Dataset(X_train , label = y_train,  free_raw_data=False)
val_df = lgb.Dataset(X_val, label = y_val,   free_raw_data=False)

In [ ]:
params = {
    "objective" : "poisson",
    "metric" : "rmse",
    "learning_rate" : 0.075,
    "sub_feature" : 0.8,
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 0.1,
    "verbosity" : 1,
    "num_iterations" : 1000,        
    "num_leaves" : 128,
    "min_data_in_leaf": 50,
}


m_lgb = lgb.train(params, train_df, valid_sets = [train_df, val_df], 
                  early_stopping_rounds = 200,
                  verbose_eval=100)

In [ ]:
params = {
    "objective" : "tweedie",
    "metric" : "rmse",
    "learning_rate" : 0.075,
    "sub_feature" : 0.8,
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 0.1,
    "verbosity" : 1,
    "num_iterations" : 1000,        
    "num_leaves" : 128,
    "min_data_in_leaf": 50,
}


m_lgb = lgb.train(params, train_df, valid_sets = [train_df, val_df], 
                  early_stopping_rounds = 200,
                  verbose_eval=100)

In [ ]:
def objective(trial):
    # Define hyperparameters
    params = {
        'feature_pre_filter' : False,
        'objective': trial.suggest_categorical("objective", ["poisson", "tweedie"]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 5e-2, 1e-1),
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq": 1,
#         "lambda_l2" : 0.1,
        'num_iterations' : trial.suggest_int('num_iterations', 800, 1100, step=100),        
        'num_leaves' : trial.suggest_int('num_leaves', 100, 128),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 30, 60, step=10),
        'metric' : 'rmse',
    }
    
    # Train model
    model = lgb.train(params, train_df, valid_sets=val_df, early_stopping_rounds=200)
    
    # Return loss on validation set
    return model.best_score['valid_0']['rmse']

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params
print(f'BEST PARAMS: {best_params}')